# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [1]:
import nltk
import numpy as np
import pandas as pd
import re 
import spacy
import warnings

from joblib import Parallel, delayed
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import (
    LogisticRegression,
    SGDClassifier
)
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    cross_val_score,
    RandomizedSearchCV
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#from sklearn.tree import DecisionTreeClassifier
from time import time
from tqdm import notebook 
from tqdm import tqdm

In [2]:
!python3 -m spacy download python -m spacy download en

Python 


In [3]:
# Отключение warning
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
#Формат float
pd.options.display.float_format = '{:,.2f}'.format
# Сброс ограничений на число столбцов
pd.options.display.max_columns = None


In [4]:
# Инициализация прогресс-баров для apply()
tqdm.pandas()

### Загрузка

In [5]:
try:
    data = pd.read_csv('datasets/toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

### Анализ датасета

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
data.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

#### Столбец 'Unnamed: 0'

In [9]:
data['Unnamed: 0'].describe()

count   159,292.00
mean     79,725.70
std      46,028.84
min           0.00
25%      39,872.75
50%      79,721.50
75%     119,573.25
max     159,450.00
Name: Unnamed: 0, dtype: float64

In [10]:
data.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [11]:
len(data['Unnamed: 0'].unique())

159292

Похоже, что столбец 'Unnamed: 0' - лишний, он содержит просто номер сообщения.<br />
Он не совпадает с индексами - видимо, это старый индекс. Похоже, часть сообщений была удалена, после чего индекс был сброшен.<br />
Этот столбец не содержит полезной информации - стоит его удалить.

In [12]:
data = data.drop('Unnamed: 0', axis = 1)

#### Столбец 'text'

In [13]:
data['text'].describe()

count                                                159292
unique                                               159292
top       Explanation\nWhy the edits made under my usern...
freq                                                      1
Name: text, dtype: object

Столбец содержит текстовые сообщения. Пропуски отсутствуют, все сообщения уникальные

#### Столбец 'toxic'

Целевой признак

In [14]:
data['toxic'].describe()

count   159,292.00
mean          0.10
std           0.30
min           0.00
25%           0.00
50%           0.00
75%           0.00
max           1.00
Name: toxic, dtype: float64

In [15]:
data['toxic'].unique()

array([0, 1], dtype=int64)

In [16]:
data['toxic'].value_counts(normalize=True)

0   0.90
1   0.10
Name: toxic, dtype: float64

Токсичных текстов - 10 %. Возможен перекос при обучении

### Подготовка данных

#### Разбивка датасета

In [17]:
features = data.drop('toxic', axis = 1)
target = data['toxic']

In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=1176, stratify = target)

In [19]:
features_train = features_train.reset_index(drop=True)
features_test = features_test.reset_index(drop=True)
target_train = target_train.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

In [20]:
target_train.value_counts(normalize=True)

0   0.90
1   0.10
Name: toxic, dtype: float64

In [21]:
features_train.shape

(127433, 1)

#### Подготовка текста

##### Вспомогательные функции

**Функция очистки текстов постов:**

In [22]:
#ввожу функцию очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

**Функция лемматизации тектов Spacy:**

In [24]:
#nlp.pipe_names

In [25]:
    disabled_pipes = [ "parser",  "ner"]
    nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

In [26]:
def spacy_lem(text):
    doc = nlp(text)
    s = []
    for token in doc:
        s.append(token.lemma_)
        s.append(token.pos_)
    return(' '.join(s))

In [27]:
batch_size = 1000

In [28]:
def spacy_batch_lem(texts,batch_size):
    res = []
    for i in notebook.tqdm(range((len(texts) // batch_size)+1)):
        batch = texts[batch_size*i:batch_size*(i+1)]
        merged_batch = "|".join(batch)
        lemmed_batch = spacy_lem(merged_batch)
        #res.append(pd.Series(lemmed_batch.split(sep="|")))
        for t in lemmed_batch.split(sep="|"):
            res.append(t)
        #print(res)
    return pd.Series(res)

##### Для трейна

In [29]:
features_train['ready_text'] = features_train['text'].progress_apply(clear_text) 

100%|███████████████████████████████████████████████████████████████████████| 127433/127433 [00:05<00:00, 22317.50it/s]


In [30]:
features_train['lemm_text'] = spacy_batch_lem(features_train['ready_text'],batch_size).reset_index(drop=True)

  0%|          | 0/128 [00:00<?, ?it/s]

##### Для теста

In [32]:
features_test['ready_text'] = features_test['text'].progress_apply(clear_text) 

100%|█████████████████████████████████████████████████████████████████████████| 31859/31859 [00:01<00:00, 22933.65it/s]


In [33]:
features_test['lemm_text'] = spacy_batch_lem(features_test['ready_text'],batch_size).reset_index(drop=True)

  0%|          | 0/32 [00:00<?, ?it/s]

In [35]:
features_train

,text,ready_text,lemm_text
0,Their destructive nature that seems to destroy...,their destructive nature that seems to destroy...,their PRON destructive ADJ nature NOUN that PR...
1,Obviously for a print work the verification of...,obviously for a print work the verification of...,obviously ADV for ADP a DET print NOUN work VE...
2,.\n\n According to TV (History Channel? NOVA?...,according to tv history channel nova the stove...,according NOUN to PART tv VERB history NOUN ch...
3,"""\n\n Edit request from , 25 October 2011 \n\n...",edit request from october incorrect statement ...,edit NOUN request NOUN from ADP october PROPN ...
4,I do not care if you block me for stating a kn...,i do not care if you block me for stating a kn...,i NOUN do AUX not PART care VERB if SCONJ you ...
...,...,...,...
127428,"""\n\n 1947-48 Palestinian Civil War \n\n3 volu...",palestinian civil war volunteers worked hard t...,palestinian ADJ civil ADJ war NOUN volunteer N...
127429,"If you are going to edit this page, ADD to it,...",if you are going to edit this page add to it d...,if ADJ you PRON be AUX go VERB to PART edit VE...
127430,YOU SHOULD NOT INTERFARE \n\n(ranbir )\ni thi...,you should not interfare ranbir i think you ar...,you NOUN should AUX not PART interfare VERB ra...
127431,The problem is that most other Senators don't ...,the problem is that most other senators don t ...,the DET problem NOUN be AUX that SCONJ most AD...


# Подготовка данных для TF-IDF

#### Загрузка стоп-слов

In [36]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Расчет корпусов

In [37]:
corpus_train = features_train['lemm_text']

In [38]:
corpus_test = features_test['lemm_text']

### Итоги этапа:

1. Загружен датасет, содержащий 159292 записи с текстами сообщений и их оценками. Наблюдается дисбаланс токсичных/нетоксичных комментариев, приблизительно 1:9
2. Датасет содержит столбец со старыми индексами записей, 'Unnamed: 0'. Данный столбец удалён.
3. Датасет был разбит на обучающую и тестовую выборку в пропорции 9:1.
4. Произведена очистка сообщений от спецсимволов.
5. Лемматизация сообщений не производилась из-за ее длительности
6. Выделен ключевые признаки и корпуса сообщений для обучающей и тестовой выборок

## Обучение

В связи с дисбалансом классов, используем модели с параметром class_weight='balanced'

In [40]:
#tqdm.sklearn()

### PipeLine для DummyClassifier

In [41]:
dummy_pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', DummyClassifier()),
])

In [42]:
dummy_parameters = {
    'clf__strategy':('stratified', 'prior', 'uniform', 'most_frequent'),
}

In [43]:
rs = RandomizedSearchCV(dummy_pipeline, 
                        dummy_parameters, 
                        n_iter = 5, 
                        scoring = 'f1',
                        cv = 5, 
                        verbose = False, 
                        n_jobs=-1, 
                        random_state=1176)

In [44]:
%%time
start = time()
rs.fit(corpus_train,target_train);
dummy_time = time() - start
dummy_score = rs.best_score_
dummy_model = rs.best_estimator_
dummy_params = rs.best_params_

CPU times: total: 21.8 s
Wall time: 1min 31s


In [45]:
print('Значение F1: {:.2f}'.format(dummy_score))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(dummy_time))
print('Гиперпараметры модели:', dummy_params)

Значение F1: 0.17
Время подбора гиперпараметров: 91.99 сек.
Гиперпараметры модели: {'clf__strategy': 'uniform'}


### PipeLine для LogisticRegression

In [46]:
line_pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', LogisticRegression(random_state=1176, solver='liblinear', class_weight='balanced')),
])

In [47]:
line_parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__norm': ('l1', 'l2'),
}

In [48]:
rs = RandomizedSearchCV(line_pipeline, 
                        line_parameters, 
                        n_iter = 5, 
                        scoring = 'f1',
                        cv = 5, 
                        verbose = False, 
                        n_jobs=-1, 
                        random_state=1176)

In [49]:
%%time
start = time()
rs.fit(corpus_train,target_train);
linear_time = time() - start
linear_score = abs(rs.best_score_)
linear_model = rs.best_estimator_
linear_params = rs.best_params_

CPU times: total: 1min 23s
Wall time: 3min 7s


In [50]:
print('Значение F1: {:.2f}'.format(linear_score))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(linear_time))
print('Гиперпараметры модели:', linear_params)

Значение F1: 0.74
Время подбора гиперпараметров: 187.54 сек.
Гиперпараметры модели: {'vect__norm': 'l2', 'vect__ngram_range': (1, 2)}


### PipeLine для LogisticRegression со смещенным порогом

In [51]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
   corpus_train,target_train, test_size=0.2, random_state=1176, stratify = target_train)

In [52]:
line_pipeline_2 = Pipeline([
    ('vect', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', LogisticRegression(random_state=1176, solver='liblinear', class_weight='balanced')),
])

In [53]:
%%time
line_pipeline_2.fit(features_train_2, target_train_2)

CPU times: total: 23.5 s
Wall time: 15.5 s


Pipeline(steps=[('vect',
                 TfidfVectorizer(stop_words=["it's", 'he', 'too', "weren't",
                                             'above', 'was', 'themselves',
                                             'into', 'just', 'won', 'any', 'if',
                                             "mightn't", 'once', 'where', 'out',
                                             're', "wasn't", 'or', 'who', 'll',
                                             'we', 'there', 'hadn', 'what',
                                             'doing', 'by', "should've",
                                             "isn't", 'all', ...])),
                ('clf',
                 LogisticRegression(class_weight='balanced', random_state=1176,
                                    solver='liblinear'))])

In [54]:
start = time()
probabilities_valid = line_pipeline_2.predict_proba(features_valid_2)
probabilities_one_valid = probabilities_valid[:, 1]

linear_2_score = 0
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold # < напишите код здесь >
    f1 = f1_score(target_valid_2,predicted_valid) # < напишите код здесь >
    if f1 > linear_2_score:
        linear_2_score = f1
        best_trh = threshold
linear_2_time = time() - start

In [55]:
print('Значение F1: {:.2f} при пороге = {:.2f}'.format(linear_2_score, best_trh))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(linear_2_time))

Значение F1: 0.78 при пороге = 0.68
Время подбора гиперпараметров: 3.62 сек.


### PipeLine для SGDClassifier

Согласно scikit-learn.org:

In [56]:
max_iterate = 10**6 // len(corpus_train)+1
max_iterate

8

In [57]:
sgd_pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', SGDClassifier(max_iter=max_iterate, random_state=1176,class_weight='balanced')),
])

In [58]:
sgd_parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__penalty':('l1','l2','elasticnet'),
    
}

In [59]:
rs = RandomizedSearchCV(sgd_pipeline, 
                        sgd_parameters, 
                        n_iter = 5, 
                        scoring = 'f1',
                        cv = 5, 
                        verbose = False, 
                        n_jobs=-1, 
                        random_state=1176)

In [60]:
%%time
start = time()
rs.fit(corpus_train,target_train);
sgd_time = time() - start
sgd_score = abs(rs.best_score_)
sgd_model = rs.best_estimator_
sgd_params = rs.best_params_

CPU times: total: 26 s
Wall time: 2min 16s


In [61]:
print('Значение F1: {:.2f}'.format(sgd_score))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(sgd_time))
print('Гиперпараметры модели:', sgd_params)

Значение F1: 0.72
Время подбора гиперпараметров: 136.44 сек.
Гиперпараметры модели: {'vect__ngram_range': (1, 1), 'clf__penalty': 'l1'}


### PipeLine для LightGBM

In [62]:
lgbm_pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', LGBMClassifier(class_weight = 'balanced')),
])

In [63]:
n_estimators = [int(x) for x in np.linspace(start = 51, stop = 151, num = 10)]
max_depth = [int(x) for x in np.linspace(start = 1, stop = 30, num = 5)]
learning_rate = [0.25, 0.5, 0.75]

In [64]:
lgbm_parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__n_estimators': n_estimators,
    'clf__max_depth':max_depth,
    'clf__learning_rate': learning_rate,
}

In [65]:
lgbm_parameters

{'vect__ngram_range': ((1, 1), (1, 2)),
 'clf__n_estimators': [51, 62, 73, 84, 95, 106, 117, 128, 139, 151],
 'clf__max_depth': [1, 8, 15, 22, 30],
 'clf__learning_rate': [0.25, 0.5, 0.75]}

In [66]:
rs = RandomizedSearchCV(lgbm_pipeline, 
                        lgbm_parameters, 
                        n_iter = 5, 
                        scoring = 'f1',
                        cv = 5, 
                        verbose = False, 
                        n_jobs=-1, 
                        random_state=1176)

In [67]:
%%time
start = time()
rs.fit(corpus_train,target_train);
lgbm_time = time() - start
lgbm_score = abs(rs.best_score_)
lgbm_model = rs.best_estimator_
lgbm_params = rs.best_params_

CPU times: total: 12min 22s
Wall time: 25min 46s


In [68]:
print('Значение F1: {:.2f}'.format(lgbm_score))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(lgbm_time))
print('Гиперпараметры модели:', lgbm_params)

Значение F1: 0.75
Время подбора гиперпараметров: 1546.09 сек.
Гиперпараметры модели: {'vect__ngram_range': (1, 2), 'clf__n_estimators': 117, 'clf__max_depth': 22, 'clf__learning_rate': 0.5}


### Итоги этапа:

In [69]:
print('Наилучший результат на обучающей выборке показала модель "PipeLine для LogisticRegression со смещенным порогом" с параметрами:')
print('Значение F1: {:.2f} при пороге = {:.2f}'.format(linear_2_score, best_trh))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(linear_2_time))

Наилучший результат на обучающей выборке показала модель "PipeLine для LogisticRegression со смещенным порогом" с параметрами:
Значение F1: 0.78 при пороге = 0.68
Время подбора гиперпараметров: 3.62 сек.


Незначительно от неё отстаёт:

In [71]:
print('"PipeLine для LightGBM"')
print('Значение F1: {:.2f}'.format(lgbm_score))
print('Время подбора гиперпараметров: {:.2f} сек.'.format(lgbm_time))
print('Гиперпараметры модели:', lgbm_params)

"PipeLine для LightGBM"
Значение F1: 0.75
Время подбора гиперпараметров: 1546.09 сек.
Гиперпараметры модели: {'vect__ngram_range': (1, 2), 'clf__n_estimators': 117, 'clf__max_depth': 22, 'clf__learning_rate': 0.5}


Модель "PipeLine для SGDClassifier" и "PipeLine для LogisticRegression" для решения данной задачи не подходят из-за невыполнения требования значения метрики

## Тестирование

Проверим наилучшую модель на тестовой выборке

In [72]:
probabilities_test = line_pipeline_2.predict_proba(corpus_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > best_trh
f1 = f1_score(target_test,predicted_test)
print("F1 = {:.2f}".format(f1))

F1 = 0.78


## Выводы

In [73]:
print('  1. Наилучший результат на обучающей выборке показала модель "Логистическая регрессия со смещенным порогом"')
print('   Значение F1 при пороге = {:.2f}'.format(best_trh))
print('   - на обучающей выборке: {:.2f}'.format(linear_2_score))
print('   - на тестовой выборке: {:.2f}'.format(f1))
print('   Время подбора гиперпараметров: {:.2f} сек.'.format(linear_time))

  1. Наилучший результат на обучающей выборке показала модель "Логистическая регрессия со смещенным порогом"
   Значение F1 при пороге = 0.68
   - на обучающей выборке: 0.78
   - на тестовой выборке: 0.78
   Время подбора гиперпараметров: 187.54 сек.


2. Значение метрики F1 данной модели на обучающей и тестовой выборках выше требуемого значения 0.75, следовательно, данная модель подходит для "боевого" применения.

3. При проведении предварительной лемматизации текста возможно улучшить значение метрики модели, но при этом ощутимо ухудшится время ее работы (а именно - время предварительной подготовки данных).